In [1]:
import ccxt
import pandas as pd
import datetime
import time

In [2]:
# Initialize the binance client
binance = ccxt.binance()

# Fetch all trading pairs
markets = binance.fetch_markets()

# Filter for USDT trading pairs
usd_pairs = [market['symbol'] for market in markets if market['quote'] == 'USD']

# Create an empty dataframe to stack the annual data
stack = pd.DataFrame()

# Get the current year
current_year = datetime.datetime.now().year

In [24]:
markets[0]

{'id': 'ETHBTC',
 'lowercaseId': 'ethbtc',
 'symbol': 'ETH/BTC',
 'base': 'ETH',
 'quote': 'BTC',
 'settle': None,
 'baseId': 'ETH',
 'quoteId': 'BTC',
 'settleId': None,
 'type': 'spot',
 'spot': True,
 'margin': True,
 'swap': False,
 'future': False,
 'option': False,
 'active': True,
 'contract': False,
 'linear': None,
 'inverse': None,
 'taker': 0.001,
 'maker': 0.001,
 'contractSize': None,
 'expiry': None,
 'expiryDatetime': None,
 'strike': None,
 'optionType': None,
 'precision': {'amount': 4, 'price': 5, 'base': 8, 'quote': 8},
 'limits': {'leverage': {'min': None, 'max': None},
  'amount': {'min': 0.0001, 'max': 100000.0},
  'price': {'min': 1e-05, 'max': 922327.0},
  'cost': {'min': 0.0001, 'max': 9000000.0},
  'market': {'min': 0.0, 'max': 5323.26603833}},
 'info': {'symbol': 'ETHBTC',
  'status': 'TRADING',
  'baseAsset': 'ETH',
  'baseAssetPrecision': '8',
  'quoteAsset': 'BTC',
  'quotePrecision': '8',
  'quoteAssetPrecision': '8',
  'baseCommissionPrecision': '8',
  '

In [25]:
[market['symbol'] for market in markets if market['base'] == 'USDT']

['USDT/TRY',
 'USDT/RUB',
 'USDT/ZAR',
 'USDT/IDRT',
 'USDT/UAH',
 'USDT/BIDR',
 'USDT/BKRW',
 'USDT/DAI',
 'USDT/NGN',
 'USDT/BRL',
 'USDT/BVND',
 'USDT/PLN',
 'USDT/RON',
 'USDT/ARS',
 'USDT/VAI']

In [19]:
types = set(market['base'] for market in markets)
print(types)

{'SUSHIUP', 'XTZDOWN', 'IRIS', 'SKY', 'PHA', 'BNBDOWN', 'TNT', 'XRPBULL', 'DAR', 'AGI', 'MCO', 'TORN', '1000XEC', 'COMP', 'EOSDOWN', 'ARKM', 'AERGO', 'FUEL', 'NKN', 'POA', 'BTTC', 'SSV', 'CLOAK', 'BLUEBIRD', 'REQ', 'ETH', 'SUSHIDOWN', 'AVAX', 'ETHDOWN', 'ONG', 'ANT', 'IDEX', 'ALCX', 'ORN', 'RAD', 'BKRW', 'REEF', 'FLOW', 'WTC', 'STORM', 'ILV', 'KP3R', 'BSW', 'WRX', 'STPT', 'PEPE', 'NCASH', 'ALICE', 'LUN', 'GLM', 'LOKA', 'MLN', 'RSR', 'LINKUP', 'PROM', 'FDUSD', 'CYBER', 'CELR', 'BULL', 'CFX', 'EPX', 'DODOX', 'KEY', 'BUSD', 'MBOX', 'TRXUP', 'VIDT', 'XLMUP', 'PHX', 'SNGLS', 'BCH', 'FUN', '1INCHDOWN', 'DLT', 'WAXP', 'BOT', 'TRX', 'PERL', 'OAX', 'SUB', 'OGN', 'LUNA', 'GRT', 'MINA', 'QKC', 'BQX', 'ENJ', 'LAZIO', 'IOST', 'TOMO', 'ARPA', 'UNFI', 'LTO', 'APPC', 'CMT', 'NAS', 'STRAX', 'BTCST', 'MDX', 'FET', 'DUSK', 'DOT', 'NPXS', 'USDSB', 'KAVA', 'ZEC', 'NU', 'RENBTC', 'WING', 'RPL', 'UFT', 'HBAR', 'IOTA', 'HIVE', 'VEN', 'XEC', 'AUTO', 'AUDIO', 'KDA', 'EOSBEAR', 'BCN', 'LPT', 'ALGO', 'EDO', 'NANO

In [21]:
filtered_markets[0]

{'id': 'USDTTRY',
 'lowercaseId': 'usdttry',
 'symbol': 'USDT/TRY',
 'base': 'USDT',
 'quote': 'TRY',
 'settle': None,
 'baseId': 'USDT',
 'quoteId': 'TRY',
 'settleId': None,
 'type': 'spot',
 'spot': True,
 'margin': False,
 'swap': False,
 'future': False,
 'option': False,
 'active': True,
 'contract': False,
 'linear': None,
 'inverse': None,
 'taker': 0.001,
 'maker': 0.001,
 'contractSize': None,
 'expiry': None,
 'expiryDatetime': None,
 'strike': None,
 'optionType': None,
 'precision': {'amount': 0, 'price': 2, 'base': 8, 'quote': 8},
 'limits': {'leverage': {'min': None, 'max': None},
  'amount': {'min': 1.0, 'max': 92141578.0},
  'price': {'min': 0.01, 'max': 1000.0},
  'cost': {'min': 10.0, 'max': 90000000.0},
  'market': {'min': 0.0, 'max': 2743835.02916666}},
 'info': {'symbol': 'USDTTRY',
  'status': 'TRADING',
  'baseAsset': 'USDT',
  'baseAssetPrecision': '8',
  'quoteAsset': 'TRY',
  'quotePrecision': '8',
  'quoteAssetPrecision': '8',
  'baseCommissionPrecision': '8

In [4]:
markets[0]

{'id': 'ETHBTC',
 'lowercaseId': 'ethbtc',
 'symbol': 'ETH/BTC',
 'base': 'ETH',
 'quote': 'BTC',
 'settle': None,
 'baseId': 'ETH',
 'quoteId': 'BTC',
 'settleId': None,
 'type': 'spot',
 'spot': True,
 'margin': True,
 'swap': False,
 'future': False,
 'option': False,
 'active': True,
 'contract': False,
 'linear': None,
 'inverse': None,
 'taker': 0.001,
 'maker': 0.001,
 'contractSize': None,
 'expiry': None,
 'expiryDatetime': None,
 'strike': None,
 'optionType': None,
 'precision': {'amount': 4, 'price': 5, 'base': 8, 'quote': 8},
 'limits': {'leverage': {'min': None, 'max': None},
  'amount': {'min': 0.0001, 'max': 100000.0},
  'price': {'min': 1e-05, 'max': 922327.0},
  'cost': {'min': 0.0001, 'max': 9000000.0},
  'market': {'min': 0.0, 'max': 5323.26603833}},
 'info': {'symbol': 'ETHBTC',
  'status': 'TRADING',
  'baseAsset': 'ETH',
  'baseAssetPrecision': '8',
  'quoteAsset': 'BTC',
  'quotePrecision': '8',
  'quoteAssetPrecision': '8',
  'baseCommissionPrecision': '8',
  '

In [5]:
len(usd_pairs)

62

In [6]:
for pair in usd_pairs:
    for year in range(2017, current_year+1):
        start_date = datetime.datetime(year, 1, 1)
        end_date = datetime.datetime(year, 12, 31)
        
        while start_date <= end_date:
            current_end_date = start_date + datetime.timedelta(days=199)
            if current_end_date > end_date:
                current_end_date = end_date
            
            # Convert start and end dates to timestamps
            since = binance.parse8601(start_date.strftime('%Y-%m-%dT%H:%M:%S.%fZ'))
            till = binance.parse8601(current_end_date.strftime('%Y-%m-%dT%H:%M:%S.%fZ'))
            
            # Fetch the klines data
            klines = binance.fetch_ohlcv(pair, '1d', since, limit=(till-since)//(24*60*60*1000)+1)
            
            # Convert the klines data to a dataframe and append to stack
            headers = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
            df = pd.DataFrame(klines, columns=headers)
            
            # Convert the 'timestamp' column to a date format
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.rename(columns={'timestamp': 'date'}, inplace=True)
            
            # Add a column for the trading pair
            df['pair'] = pair
            
            stack = pd.concat([stack, df])
            
            # Move to the next chunk of data
            start_date += datetime.timedelta(days=200)
            
            # Introduce a delay to avoid hitting rate limits
            time.sleep(1)


In [16]:
stack

,date,open,high,low,close,volume,pair
0,2020-08-11,11785.00,11785.00,11125.50,11393.50,4977.814598,BTC/USD:BTC
1,2020-08-12,11393.90,11628.30,11122.60,11580.60,5378.780166,BTC/USD:BTC
2,2020-08-13,11581.60,11808.10,11278.90,11795.60,6901.253226,BTC/USD:BTC
3,2020-08-14,11796.70,11873.80,11652.70,11777.10,5854.894246,BTC/USD:BTC
4,2020-08-15,11781.10,11999.80,11689.50,11867.80,6803.053490,BTC/USD:BTC
...,...,...,...,...,...,...,...
61,2023-09-19,211.76,215.72,211.21,213.23,2532.597502,BNB/USD:BNB-231229
62,2023-09-20,213.26,213.99,209.74,210.35,2018.930448,BNB/USD:BNB-231229
63,2023-09-21,210.47,211.53,206.16,207.83,2926.268625,BNB/USD:BNB-231229
64,2023-09-22,207.80,209.13,206.72,208.37,1124.828508,BNB/USD:BNB-231229


In [18]:
# 여기에 stack 변수가 정의되어 있다고 가정하겠습니다.
stack['pair'].unique()

array(['BTC/USD:BTC', 'BTC/USD:BTC-230929', 'BTC/USD:BTC-231229',
       'ETH/USD:ETH', 'ETH/USD:ETH-230929', 'ETH/USD:ETH-231229',
       'LINK/USD:LINK', 'BNB/USD:BNB', 'TRX/USD:TRX', 'DOT/USD:DOT',
       'ADA/USD:ADA', 'EOS/USD:EOS', 'LTC/USD:LTC', 'BCH/USD:BCH',
       'XRP/USD:XRP', 'ETC/USD:ETC', 'FIL/USD:FIL', 'EGLD/USD:EGLD',
       'DOGE/USD:DOGE', 'UNI/USD:UNI', 'THETA/USD:THETA', 'XLM/USD:XLM',
       'SOL/USD:SOL', 'FTM/USD:FTM', 'SAND/USD:SAND', 'MANA/USD:MANA',
       'AVAX/USD:AVAX', 'GALA/USD:GALA-221226', 'MATIC/USD:MATIC',
       'NEAR/USD:NEAR', 'ATOM/USD:ATOM', 'AAVE/USD:AAVE', 'AXS/USD:AXS',
       'ROSE/USD:ROSE', 'XTZ/USD:XTZ', 'ICX/USD:ICX', 'ALGO/USD:ALGO',
       'RUNE/USD:RUNE', 'APE/USD:APE', 'VET/USD:VET-221226',
       'ZIL/USD:ZIL-221226', 'KNC/USD:KNC', 'XMR/USD:XMR', 'GMT/USD:GMT',
       'OP/USD:OP', 'ENS/USD:ENS', 'CHZ/USD:CHZ', 'APT/USD:APT',
       'ADA/USD:ADA-230929', 'LINK/USD:LINK-230929', 'BCH/USD:BCH-230929',
       'DOT/USD:DOT-230929', 'XRP

In [6]:
# for pair in usd_pairs:
#     for year in range(2017, current_year+1):
#         # Convert start and end dates to timestamps
#         since = binance.parse8601(f'{year}-01-01T00:00:00Z')
#         till = binance.parse8601(f'{year}-12-31T23:59:59Z')
        
#         # Fetch the klines data
#         klines = binance.fetch_ohlcv(pair, '1d', since, limit=(till-since)//(24*60*60*1000)+1)
        
#         # Convert the klines data to a dataframe and append to stack
#         headers = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
#         df = pd.DataFrame(klines, columns=headers)
        
#         # Convert the 'timestamp' column to a date format
#         df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
#         df.rename(columns={'timestamp': 'date'}, inplace=True)
        
#         # Add a column for the trading pair
#         df['pair'] = pair
        
#         stack = pd.concat([stack, df])
    
#     # Introduce a delay to avoid hitting rate limits
#     time.sleep(1) 

ExchangeError: binance {"code":-4088,"msg":"Maximum time interval is 200 days."}

In [15]:
stack[stack['pair'] == 'BTC/USD']

,date,open,high,low,close,volume,pair


In [26]:
stack.to_csv("cryptos_usd.csv", index=False)

In [3]:
data = pd.read_csv("all_cryptos_usdt.csv")

In [4]:
data.head()

,date,open,high,low,close,volume,pair
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,BTC/USDT
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,BTC/USDT
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,BTC/USDT
3,2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,BTC/USDT
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,BTC/USDT
